In [2]:
from utils import *
from word2vec import *
from vocab import *
import tensorflow as tf
from data_utils import *
from tensorflow.contrib import rnn

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.202 seconds.
Prefix dict has been built succesfully.


In [3]:
_BATCH_SIZE = 10
_NUM_LAYERS = 2

In [55]:
embedding = tf.Variable(tf.constant(0.0, shape=[VOCAB_SIZE, NUM_UNITS]), trainable = False)

In [5]:
embed_ph = get_word_embedding(NUM_UNITS)

In [6]:
embed_init = embedding.assign(embed_ph)

In [7]:
inputs = tf.placeholder(tf.int32, [_BATCH_SIZE, None])

In [8]:
input_lengths = tf.placeholder(tf.int32, [_BATCH_SIZE])

In [9]:
lstm_cell = rnn.BasicLSTMCell(NUM_UNITS)

In [10]:
init_state = lstm_cell.zero_state(_BATCH_SIZE, dtype = tf.float32)

In [13]:
def get_names(fn):
    with open(fn) as f:
        names = f.readlines()

    return [n.strip().decode('utf8') for n in names]

In [14]:
male_file = "./data/name/male.txt"
female_file = "./data/name/female.txt"

In [15]:
male_names = get_names(male_file)
female_names = get_names(female_file)

In [17]:
names = ([(n, 'm') for n in male_names if len(n) < 5] +
         [(n, 'f') for n in female_names if len(n) < 5])
random.shuffle(names)
print "names len = %d" % len(names)

names len = 37020


In [20]:
names[0][0]

u'\u8d3e\u5efa\u534e'

# Train

In [23]:
_, ch2int = get_vocab()

In [24]:
line = names[0][0]

In [42]:
toks = line.strip()
batch_s = ([0]+[ch2int[ch] for ch in toks[-2:]]+[VOCAB_SIZE-1])

In [58]:
emb_batch_inputs = [tf.nn.embedding_lookup(embedding, x)
                              for x in batch_s]
for layer_i in xrange(_NUM_LAYERS):
    cell_fw = tf.contrib.rnn.LSTMCell(
          NUM_UNITS,
          initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=123),
          state_is_tuple=False)
    cell_bw = tf.contrib.rnn.LSTMCell(
          NUM_UNITS,
          initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=113),
          state_is_tuple=False)
    (emb_batch_inputs, fw_state, bw_state) = tf.contrib.rnn.static_bidirectional_rnn(
          cell_fw, cell_bw, emb_batch_inputs, dtype=tf.float32,
          sequence_length=input_lengths)
outputs = emb_batch_inputs

ValueError: Shape (1024,) must have rank 2

In [57]:
emb_batch_inputs

[<tf.Tensor 'embedding_lookup_11:0' shape=(1024,) dtype=float64>,
 <tf.Tensor 'embedding_lookup_12:0' shape=(1024,) dtype=float64>,
 <tf.Tensor 'embedding_lookup_13:0' shape=(1024,) dtype=float64>,
 <tf.Tensor 'embedding_lookup_14:0' shape=(1024,) dtype=float64>]

In [63]:
tf.nn.embedding_lookup(embedding, 1417)

<tf.Tensor 'embedding_lookup_23:0' shape=(1024,) dtype=float32>

In [54]:
embedding

array([[-0.92823611,  0.19396231, -0.59372165, ..., -0.56219579,
        -0.21609338,  0.79316901],
       [ 0.26942933, -0.52706224,  0.2432324 , ..., -0.15089326,
        -0.01310407, -0.33053225],
       [-0.26546228,  0.73082376,  0.20869176, ..., -0.47377276,
        -0.13454807, -0.33626828],
       ..., 
       [-0.65682134, -0.95453682,  0.4192832 , ...,  0.32635209,
         0.96977321,  0.76176611],
       [-0.49038366,  0.05002729, -0.78279834, ...,  0.82635526,
         0.8577958 ,  0.60134101],
       [-0.14254919,  0.17257579, -0.6304464 , ...,  0.64196031,
         0.94730638,  0.69286899]])